# Chatbot with Groq

This notebook demonstrates how to build a simple chatbot using Groq's API.

## Setup Instructions:

1. **Get a Groq API key:**
   - Go to https://console.groq.com/
   - Sign up or log in
   - Go to API Keys section
   - Create a new API key

2. **Set your API key in Cell 2**

3. **Run the cells in order!**

---


In [1]:
# Install required packages
%pip install groq python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from groq import Groq

# Initialize Groq client
load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

print("✅ Groq client initialized!")


✅ Groq client initialized!


## Simple Chatbot Function

This chatbot is designed to be an expert baker assistant that helps with sourdough bread questions.

In [3]:
def chat_with_bot(user_input, model="llama-3.3-70b-versatile"):
    """
    Send a message to the chatbot and get a response.

    Args:
        user_input: The user's message
        model: Groq model to use (default: llama-3.3-70b-versatile)

    Returns:
        str: The chatbot's response
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system", 
                    "content": "You are an expert baker assistant that helps home bakers with questions about sourdough bread dough."
                },
                {
                    "role": "user", 
                    "content": user_input
                }
            ],
            temperature=0.7,
            max_tokens=500
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

print("✅ Chatbot function defined!")


✅ Chatbot function defined!


## Test the Chatbot

Let's try asking the chatbot some questions about sourdough bread!

In [4]:
# Test question 1
user_input = "When is starter ready to use?"
response = chat_with_bot(user_input)
print(f"You: {user_input}")
print(f"\nChatbot: {response}")

You: When is starter ready to use?

Chatbot: Your sourdough starter is the heart of your bread-making process. It's ready to use when it has reached a state of maturity and stability, which usually takes around 7-14 days of regular feeding and care. Here are some signs to look out for to determine if your starter is ready to use:

1. **Active Bubbles**: A healthy, active starter will have plenty of bubbles on its surface and throughout the mixture. This indicates that the wild yeast and bacteria are fermenting the sugars and producing carbon dioxide.
2. **Doubled in Size**: A mature starter will roughly double in size within 4-6 hours after feeding. This is a sign that the microorganisms are actively consuming the sugars and producing CO2.
3. **Tangy Aroma**: A sourdough starter will have a tangy, slightly sour smell, which is a result of the lactic acid produced by the bacteria.
4. **Consistent Behavior**: A mature starter will have a consistent behavior, meaning it will respond predi

## Interactive Chat Loop

Run this cell to have an interactive conversation with the chatbot. Type 'exit' or 'quit' to end the conversation.

**Note:** This will only work in a regular Jupyter notebook, not in JupyterLab or some cloud environments.

In [ ]:
# Interactive chat loop
print("Chatbot: Hello! I'm your sourdough bread assistant. Ask me anything about sourdough baking!")
print("(Type 'exit' or 'quit' to end the conversation)\n")

while True:
    try:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("\nChatbot: Happy baking!")
            break
        
        response = chat_with_bot(user_input)
        print(f"\nChatbot: {response}\n")
    except KeyboardInterrupt:
        print("\n\nChatbot: Happy baking!")
        break
    except Exception as e:
        print(f"\nError: {e}\n")

## Bonus: Conversation History

The above chatbot doesn't remember previous messages. Here's an improved version that maintains conversation history:

In [ ]:
class Chatbot:
    """
    A chatbot with conversation memory.
    """
    def __init__(self, system_prompt=None, model="llama-3.3-70b-versatile"):
        self.model = model
        self.conversation_history = []

        # Set system prompt
        if system_prompt is None:
            system_prompt = "You are an expert baker assistant that helps home bakers with questions about sourdough bread dough."

        self.conversation_history.append({
            "role": "system",
            "content": system_prompt
        })

    def chat(self, user_message):
        """
        Send a message and get a response while maintaining conversation history.
        """
        # Add user message to history
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })

        try:
            # Get response from Groq
            response = client.chat.completions.create(
                model=self.model,
                messages=self.conversation_history,
                temperature=0.7,
                max_tokens=500
            )

            # Extract assistant's message
            assistant_message = response.choices[0].message.content

            # Add assistant's response to history
            self.conversation_history.append({
                "role": "assistant",
                "content": assistant_message
            })

            return assistant_message

        except Exception as e:
            return f"Error: {str(e)}"

    def reset(self):
        """Reset conversation history."""
        system_message = self.conversation_history[0]
        self.conversation_history = [system_message]
        print("Conversation history reset!")

print("✅ Chatbot class defined!")


In [ ]:
# Create a chatbot with memory
bot = Chatbot()

# First question
response1 = bot.chat("What's the ideal temperature for proofing sourdough?")
print("You: What's the ideal temperature for proofing sourdough?")
print(f"Bot: {response1}\n")

# Follow-up question (references previous context)
response2 = bot.chat("What if my kitchen is colder than that?")
print("You: What if my kitchen is colder than that?")
print(f"Bot: {response2}")

In [ ]:
# You can see the full conversation history
print("Full conversation history:")
for msg in bot.conversation_history:
    role = msg['role'].capitalize()
    content = msg['content'][:100] + "..." if len(msg['content']) > 100 else msg['content']
    print(f"\n{role}: {content}")

## Integrate Meta Prompting

Ask your model to become its own content moderator!

**Meta prompting** is a technique where you use the LLM to evaluate and moderate its own inputs and outputs. This creates a safety layer that can:

- Filter harmful user inputs before processing
- Check generated responses before returning them to users
- Enforce content policies without hard-coded rules

### The Challenge:

1. Design a prompt that asks your model to identify whether or not a prompt or response contains harmful content.
   * Define a specific kind of "harmful content":
     * e.g. "copyright infringement"
     * e.g. "racist language"
     * e.g. "mature themes not appropriate for children"

2. Before making any prompts and after generating any responses:
   * Inject the prompt or content to your meta prompt
   * Send that to the model
   * Parse the "yes" or "no" into a boolean
   * Reject the prompt or response if the model said you should

### Solution: Content Moderation Function

First, let's create a function that checks if content contains harmful material:

In [ ]:
def check_content_safety(content, content_type="message", model="llama-3.1-8b-instant"):
    """
    Use the LLM to check if content contains harmful material.
    
    Args:
        content: The text to check (user input or bot response)
        content_type: Either "user input" or "bot response" for context
        model: Model to use for moderation (smaller/faster model recommended)
    
    Returns:
        tuple: (is_safe: bool, reason: str)
    """
    
    # Define what types of harmful content to check for
    # You can customize this list for your specific use case
    meta_prompt = f"""You are a content moderation system. Your job is to determine if the following {content_type} contains harmful content.

Check for the following types of harmful content:
1. Hate speech or discriminatory language (racism, sexism, etc.)
2. Explicit violence or threats
3. Adult/sexual content not appropriate for general audiences
4. Instructions for illegal activities
5. Personal attacks or harassment

Content to evaluate:
\"\"\"
{content}
\"\"\"

Respond with ONLY one of these two options:
- "SAFE" if the content is acceptable
- "UNSAFE: [brief reason]" if the content is harmful

Your response:"""

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a content moderation assistant. Be strict but fair."},
                {"role": "user", "content": meta_prompt}
            ],
            temperature=0.1,  # Low temperature for consistent moderation
            max_tokens=100
        )
        
        result = response.choices[0].message.content.strip().upper()
        
        if result.startswith("SAFE"):
            return True, "Content is safe"
        elif result.startswith("UNSAFE"):
            reason = result.replace("UNSAFE:", "").replace("UNSAFE", "").strip()
            return False, reason if reason else "Content flagged as potentially harmful"
        else:
            # If unclear, err on the side of caution
            return True, "Content appears safe"
            
    except Exception as e:
        # If moderation fails, log error but allow content (fail open)
        # In production, you might want to fail closed instead
        print(f"Moderation error: {e}")
        return True, f"Moderation check failed: {e}"

print("✅ Content moderation function defined!")

In [ ]:
# Test the moderation function with safe content
safe_message = "How long should I proof my sourdough bread?"
is_safe, reason = check_content_safety(safe_message, "user input")
print(f"Message: '{safe_message}'")
print(f"Is Safe: {is_safe}")
print(f"Reason: {reason}")
print()

# Test with potentially problematic content
unsafe_message = "I hate everyone who doesn't like sourdough bread, they should all suffer!"
is_safe, reason = check_content_safety(unsafe_message, "user input")
print(f"Message: '{unsafe_message}'")
print(f"Is Safe: {is_safe}")
print(f"Reason: {reason}")

### Moderated Chatbot Class

Now let's create a chatbot that automatically moderates both user inputs AND its own responses:

In [ ]:
class ModeratedChatbot:
    """
    A chatbot with conversation memory AND content moderation.
    Uses meta-prompting to check both user inputs and bot responses.
    """
    
    def __init__(self, system_prompt=None, model="llama-3.3-70b-versatile", 
                 moderation_model="llama-3.1-8b-instant"):
        self.model = model
        self.moderation_model = moderation_model
        self.conversation_history = []
        self.moderation_log = []  # Track moderation decisions
        
        # Set system prompt
        if system_prompt is None:
            system_prompt = "You are an expert baker assistant that helps home bakers with questions about sourdough bread."
        
        self.conversation_history.append({
            "role": "system",
            "content": system_prompt
        })
    
    def _moderate_content(self, content, content_type):
        """Internal method to check content safety."""
        is_safe, reason = check_content_safety(
            content, 
            content_type, 
            model=self.moderation_model
        )
        
        # Log the moderation decision
        self.moderation_log.append({
            "content_type": content_type,
            "content_preview": content[:100] + "..." if len(content) > 100 else content,
            "is_safe": is_safe,
            "reason": reason
        })
        
        return is_safe, reason
    
    def chat(self, user_message):
        """
        Send a message and get a response with moderation on both ends.
        """
        # STEP 1: Moderate the user's input BEFORE processing
        input_safe, input_reason = self._moderate_content(user_message, "user input")
        
        if not input_safe:
            return f"⚠️ I can't process that message. Reason: {input_reason}"
        
        # STEP 2: Add user message to history and get response
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })
        
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=self.conversation_history,
                temperature=0.7,
                max_tokens=500
            )
            
            assistant_message = response.choices[0].message.content
            
            # STEP 3: Moderate the bot's response BEFORE returning it
            output_safe, output_reason = self._moderate_content(assistant_message, "bot response")
            
            if not output_safe:
                # Remove the user message from history since we're not completing this exchange
                self.conversation_history.pop()
                return f"⚠️ I generated a response but it was flagged by moderation. Please rephrase your question."
            
            # STEP 4: If response is safe, add to history and return
            self.conversation_history.append({
                "role": "assistant",
                "content": assistant_message
            })
            
            return assistant_message
            
        except Exception as e:
            self.conversation_history.pop()  # Remove failed user message
            return f"Error: {str(e)}"
    
    def get_moderation_log(self):
        """Return the moderation log for inspection."""
        return self.moderation_log
    
    def reset(self):
        """Reset conversation history and moderation log."""
        system_message = self.conversation_history[0]
        self.conversation_history = [system_message]
        self.moderation_log = []
        print("Conversation and moderation log reset!")

print("✅ ModeratedChatbot class defined!")

### Test the Moderated Chatbot

In [ ]:
# Create a moderated chatbot
moderated_bot = ModeratedChatbot()

# Test with a safe question
print("Test 1: Safe question")
print("-" * 40)
response = moderated_bot.chat("What's the best flour for sourdough?")
print(f"User: What's the best flour for sourdough?")
print(f"Bot: {response[:300]}...")  # Truncate for readability

In [ ]:
# Test with a potentially problematic message
print("\nTest 2: Potentially problematic input")
print("-" * 40)
response = moderated_bot.chat("I hate people who use bread machines, they're all idiots who deserve bad bread!")
print(f"User: I hate people who use bread machines, they're all idiots who deserve bad bread!")
print(f"Bot: {response}")

In [ ]:
# View the moderation log to see what was checked
print("Moderation Log:")
print("=" * 50)
for i, entry in enumerate(moderated_bot.get_moderation_log()):
    print(f"\n[{i+1}] Type: {entry['content_type']}")
    print(f"    Preview: {entry['content_preview']}")
    print(f"    Safe: {entry['is_safe']}")
    print(f"    Reason: {entry['reason']}")

### Interactive Moderated Chat

Run this cell for an interactive chat with content moderation enabled:

In [ ]:
# Interactive moderated chat loop
# Uncomment and run to start chatting

# moderated_bot.reset()  # Start fresh
# 
# print("🛡️ Moderated Chatbot")
# print("Content moderation is enabled for both inputs and outputs.")
# print("Type 'exit' to quit, 'log' to see moderation history")
# print("-" * 50)
# 
# while True:
#     try:
#         user_input = input("\nYou: ")
#         
#         if user_input.lower() == "exit":
#             print("Goodbye!")
#             break
#         elif user_input.lower() == "log":
#             for entry in moderated_bot.get_moderation_log()[-5:]:  # Last 5 entries
#                 print(f"  [{entry['content_type']}] Safe={entry['is_safe']}: {entry['content_preview'][:50]}...")
#             continue
#         
#         response = moderated_bot.chat(user_input)
#         print(f"\nBot: {response}")
#         
#     except KeyboardInterrupt:
#         print("\n\nGoodbye!")
#         break

### Key Takeaways: Meta Prompting for Content Moderation

**What we learned:**

1. **Meta prompting** uses the LLM to evaluate content before/after processing
2. **Two-way moderation**: Check both user inputs AND bot responses
3. **Use a faster model** (like `llama-3.1-8b-instant`) for moderation to reduce latency
4. **Keep a log** of moderation decisions for debugging and auditing

**Best Practices:**

| Practice | Why |
|----------|-----|
| Define specific harmful content types | More accurate than generic "harmful" |
| Use low temperature (0.1) for moderation | Consistent, predictable results |
| Log all moderation decisions | Debugging and compliance |
| Fail gracefully | Don't crash if moderation fails |

**Customization Ideas:**

- Add domain-specific moderation rules (e.g., medical misinformation)
- Implement severity levels (warn vs. block)
- Add human review queue for borderline cases
- Create allow-lists for specific terms that might be falsely flagged